### Imports


In [1]:
import numpy as np
import librosa
import copy
from sklearn.decomposition import PCA
import IPython
import IPython.display as ipd
from itertools import product
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.io
from heapq import heappush
import heapq
import collections
import pandas as pd

In [2]:
root = "data/"



### Ideal Masks


In [12]:
piano,srate = librosa.load(root + "piano.wav",sr=None)
ocean,rate = librosa.load(root + "ocean.wav",sr=None)

In [13]:
IPython.display.Audio(root + "piano.wav")

In [14]:
IPython.display.Audio(root + "ocean.wav")

In [15]:
def calculate_SNR(S,Snew):
    inter = []
    for s,sdash in zip(S,Snew):
        inter.append(10*np.log10(np.sum(np.power(s,2))/np.sum(np.power(s-sdash,2))))
    SNR = np.mean(inter)
    return SNR

In [16]:
S = librosa.core.stft(piano, n_fft=1024, hop_length=512)
N = librosa.core.stft(ocean, n_fft=1024, hop_length=512)

In [17]:
S.shape,N.shape

((513, 157), (513, 157))

In [18]:
S,N = S.real,N.real

In [19]:
X = S + N

In [20]:
M = S / (S+N)

In [21]:
Mdash = np.power(np.abs(S),2) / (np.power(np.abs(S),2)+np.power(np.abs(N),2))

In [22]:
Snew = Mdash * X
Snew = Snew.real


In [23]:
recovered =librosa.istft(Snew,hop_length=512, length=S.shape[0]*S.shape[1])

In [24]:
librosa.output.write_wav(root + "recovered_piano.wav", recovered, srate)

Recovery using Mdash (question 2.4)

SNR = 5.0788105260235845 (question 2.5)

In [25]:
IPython.display.Audio(root + "recovered_piano.wav")

In [26]:
SNR = calculate_SNR(S,Snew)
SNR

5.0788105260235845

In [27]:
B = np.zeros(shape = (S.shape[0],S.shape[1]))

In [28]:
B.shape

(513, 157)

In [29]:
for i,j in product(range(S.shape[0]),range(S.shape[1])):
    if np.abs(S[i,j])>np.abs(N[i,j]):
        B[i,j]=1


In [30]:
Sibm = B * X

In [31]:
SNR = calculate_SNR(S,Sibm)
SNR

3.7607135424008287

IBM recovered sound (question 2.7)

SNR = 3.7607135424008287 (question 2.7)

The audio is recognizable. The pitch is a little different though. There is a little echo like drag as if it is coming from the background. That might be why my SNR is low.

In [32]:
recoveredibm =librosa.istft(Sibm,hop_length=512, length=S.shape[0]*S.shape[1])
librosa.output.write_wav(root + "recoveredibm_piano.wav", recoveredibm, srate)
IPython.display.Audio(root + "recoveredibm_piano.wav")